<center><h1 style="background-color: darkblue; color: white; font-size: 30px;"><b>👩🏻‍💻  Smapse Data Scrapt </b></h1></center>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create an empty DataFrame to store the data
columns = ['Link', 'Name', 'Rating', 'Logo', 'Image', 'Description', 'Description1', 'Description2','Programs', 'Subject', 'Activities','Addresses','price','location_texts','li_texts']
smapse = pd.DataFrame(columns=columns)

for i in range(701):
    print(i)
    url = f'https://smapse.com/catalog/?page={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    div_tags = soup.find_all('div', class_='camp-min__head-name-inner')
    for div in div_tags:
        # Initialize lists to store data for each entry
        name = []
        rate = []
        image = []
        description = []
        description1 = []
        description2 = []
        logo = []
        subjects = []
        activities = []
        programs = []
        price = []
        addresses = []
        location_texts = []
        li_texts = []


        
        
        links = div.find_all('a', href=True)
        for link in links:
            link1 = 'https://smapse.com/' + link['href']
            url1 = link1
            response1 = requests.get(url1)
            soup1 = BeautifulSoup(response1.content, 'html.parser')
            
            # University name
            university_name = soup1.find('h1').text.strip()
            name.append(university_name)

            # Ratings
            rating_div = soup1.find('div', class_='rating')
            rating = rating_div.text.strip() if rating_div else None
            rate.append(rating)

            # Images
            gallery_div = soup1.find('div', class_='gallery__nav swiper-wrapper')
            if gallery_div:
                img_tags = gallery_div.find_all('img')
                img_urls = ['https://smapse.com/' + img['src'] for img in img_tags]
                image.extend(img_urls)

            # Description
            description_div = soup1.find('div', class_='accordion')
            if description_div:
                h2_tags = description_div.find_all('h2', class_='accordion__title active')
                description.append(h2_tags[0].text.strip())

                ul_tags = description_div.find_all('ul')
                for ul in ul_tags:
                    li_tags = ul.find_all('li')
                    for li in li_tags:
                        description1.append(li.text.strip()) 
                    description1_list = '|'.join(description1)



                div_tags = description_div.find_all('div', itemprop='description')
                for div in div_tags:
                    p_tags = div.find_all('p')
                    for p in p_tags:
                        description2.append(p.text.strip() + ' ||| ')
                    description2_list = '|'.join(description2)


            # Logo
            logo_div = soup1.find('div', class_='side-logo')
            if logo_div:
                img_tag = logo_div.find('img', itemprop='logo')
                if img_tag:
                    logo_url = 'https://smapse.com/' + img_tag['src']
                    logo.append(logo_url)
                
            # Programs
            program_names = soup1.find_all('div', class_='program__name-info')
            for program_name in program_names:
                program = program_name.find('span', itemprop='name').text.strip()
                programs.append(program)
    
                    
            # Subjects and Activities
            subject_list = ""
            activity_list = ""
            span_boxes = soup1.find_all('div', class_='accordion__box')
            for span_box in span_boxes:
                strong_span = span_box.find('span', class_='strong')
                if strong_span:
                    if "Subject list" in strong_span.text:
                        subject_list = span_box.text.strip().replace('Subject list:', '').strip()
                        # Check if "Activity list" exists in the subject list
                        if "Activity list:" in subject_list:
                            subject_list, activity_list = subject_list.split("Activity list:", 1)
                            subject_list = subject_list.strip()
                            activity_list = activity_list.strip()
                    elif "activity" in strong_span.text:
                        activity_list = span_box.text.strip().replace('activity', '').strip()

            subjects.append(subject_list)
            activities.append(activity_list)
            
            # Country
            map_boxes = soup1.find_all('div', class_='map__box')

            # Iterate over each map box
            for map_box in map_boxes:
                # Find all anchor elements with class="map__address" within the current map box
                map_addresses = map_box.find_all('a', class_='map__address')

                # Extract the text from each anchor element and append it to the addresses list
                for address in map_addresses:
                    addresses.append(address.text.strip())
                  
            # city
            location_items = soup1.find_all('div', class_='location__item')

            for location_item in location_items:
                h_link_spans = location_item.find_all('span', class_='h-link')
                for span in h_link_spans:
                    location_texts.append(span.text.strip())
            
            # Price 
            booking_prices = soup1.find('div', class_='booking__pries-new')
            if booking_prices:
                prices_content = booking_prices.get_text(strip=True)
                price.append(prices_content) 

            #regfee
            accordion_div = soup1.find('div',  class_="accordion__item", id="school-about-tab-8")
            if accordion_div:
                li_elements = accordion_div.find_all('li')
                if li_elements:
                    li_texts = []
                    for li in li_elements:
                        li_texts.append(li.get_text(strip=True))
                    li_texts_combined = '|'.join(li_texts)

                        
            # Add data to DataFrame
            smapse.loc[len(smapse)] = [link1, name, rate, logo, image, description, description1_list , description2_list,subjects,activities,programs,addresses,price,location_texts,li_texts_combined]
            
smapse.columns = ['Link', 'Name', 'Rating', 'Logo', 'Image', 'Head',
       'ul_li', 'p_text','subject','activities','programs','address','price','location_texts','reg fee']    

In [ ]:
smapse['text'] = smapse['p_text'].astype(str) + smapse['ul_li'].astype(str)
smapse['text'] = smapse['text'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace('\\', '').replace('"', ''))
smapse['text'] = smapse['text'].str.strip("'")
smapse['text'] = smapse['text'].apply(lambda x: str(x))

In [ ]:
regions = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\regions.csv', encoding='latin-1')
school_types = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\school_types.csv', encoding='latin-1')
programs = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\programs.csv', encoding='latin-1')
counties = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\counties.csv', encoding='latin-1')
activities = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\activities.csv', encoding='latin-1')
subjects = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\subjects.csv', encoding='latin-1')
states = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\states.csv', encoding='latin-1')
sub_programs = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\sub_programs.csv', encoding='latin-1')
languages = pd.read_csv(r'C:\Users\eismayilov\Desktop\spamse\languages.csv', encoding='latin-1')

In [ ]:
regions_list = regions['name_en'].to_list()
school_types_list = school_types['name_en'].to_list()
programs_list = programs['name_en'].to_list()
counties_list = counties['name_en'].to_list()
activities_list = activities['name_en'].to_list()
subjects_list = subjects['name_en'].to_list()
states_list = states['name_en'].to_list()
sub_programs_list = sub_programs['name_en'].to_list()
languages_list = languages['name_en'].to_list()

In [ ]:
# Function to find region in text and return it
for y, i in [('school_list',school_types_list),('regions_list',regions_list),('languages_list', languages_list), ('sub_programs_list', sub_programs_list), ('states_list', states_list), ('subjects_list', subjects_list), ('activities_list', activities_list), ('counties_list', counties_list), ('programs_list', programs_list)]:
    def find_school(text):
        for school in i:
            if school.lower() in text.lower():
                return school
        return None

    # Apply the function to create a new column
    smapse[y] = smapse['text'].apply(find_school)

In [ ]:
smapse.to_excel('smapse_school_data.xlsx', index = False)

<h8><b style="color:darkblue; font-size: 18px;">Dataset Description

| Column Name                   | Description                                                                       |
|-------------------------------|-----------------------------------------------------------------------------------|
| University Link               | The URL or link associated with the university.                                   |
| University Name               | The name of the university.                                                      |
| University Rating             | The rating or ranking of the university.                                          |
| University Logo Link          | The URL or link to the logo image of the university.                              |
| University Images Link Source | The URL or link to the source of images related to the university.                |
| University Description Head   | The heading or title of the university description.                               |
| University Descriptions       | Detailed descriptions or information about the university.                        |
| Information about University  | Additional information or details about the university.                           |
| University Subject            | The main subjects or fields of study offered by the university.                   |
| University Activities         | Activities or extracurricular programs offered by the university.                 |
| University Programs           | Academic programs or courses offered by the university.                           |
| University Address            | The physical address or location of the university.                               |
| Registration Fee              | Fee required for registration or enrollment at the university.                    |
| Schools                       | Schools or faculties within the university.                                       |
| Regions                       | Regions or areas associated with the university.                                   |
| Languages                     | Languages of instruction or languages spoken at the university.                    |
| Sub Program                   | Subprograms or specialized areas of study within the university programs.         |
| Price                         | Price or cost associated with university-related services or products.            |
| Currency                      | Currency used for pricing or financial transactions.                               |
| Term                          | Term or semester information related to university programs.                       |
| Country                       | Country where the university is located.                                           |
| City                          | City where the university is located.                                              |
| University States             | Additional states or information related to the university.                        |
</b></h8>

<center><h1 style="background-color: darkblue; color: white; font-size: 30px;"><b> 🔚The End </b></h1></center>